# Google Data Center Power Trace Analysis

This colab demonstrates querying the Google data center power traces with bigquery, visualizing them with [Altair](https://altair-viz.github.io/), and analyzing them in conjunction with the 2019 Google cluster data.

**Important:** in order to be able to run the queries you will need to:

1. Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
1. [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
1. [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.

To begin with, we'll authenticate with GCP and import the python libraries necessary to execute this colab.

In [ ]:
#@title Please input your project id
import altair as alt
import numpy as np
import pandas as pd
from google.cloud import bigquery
# Provide credentials to the runtime
from google.colab import auth
from google.cloud.bigquery import magics

auth.authenticate_user()
print('Authenticated')
project_id = 'google.com:google-cluster-data' #@param {type: "string"}
# Set the default project id for %bigquery magic
magics.context.project = project_id

# Use the client to run queries constructed from a more complicated function.
client = bigquery.Client(project=project_id)


## Basic Queries

Here are some examples of using the [bigquery magic](https://cloud.google.com/python/docs/reference/bigquery/latest/index.html) to query the power traces.

First we'll calculate the average production utilization for a single power domain.


In [ ]:
%%bigquery
SELECT
  AVG(production_power_util) AS average_production_power_util
FROM `google.com:google-cluster-data`.powerdata_2019.cella_pdu10


Now let's find the minimum and maximum measured power utilization for each cell. We use bigquery [wildcard tables](https://cloud.google.com/bigquery/docs/querying-wildcard-tables) in order to conveniently query all trace tables at once.

In [ ]:
%%bigquery
SELECT
  cell,
  MIN(measured_power_util) AS minimum_measured_power_util,
  MAX(measured_power_util) AS maximum_measured_power_util
FROM `google.com:google-cluster-data.powerdata_2019.cell*`
GROUP BY cell

Modifying the previous query to also group by `pdu` gives us the maximum and minimum measured power utilization per power domain.

In [ ]:
%%bigquery
SELECT
  cell,
  pdu,
  MIN(measured_power_util) AS minimum_measured_power_util,
  MAX(measured_power_util) AS maximum_measured_power_util
FROM `google.com:google-cluster-data.powerdata_2019.cell*`
GROUP BY cell, pdu
ORDER BY maximum_measured_power_util

## Measured and Production Power over Time

We provide traces for two clusters of Google's new Medium Voltage Power Plane (MVPP) data center design, described in [the paper](https://research.google/pubs/pub49032/). Let's plot the measured and estimated production power utilization of one of these MVPPs: mvpp1. We'll limit this visualization to the first 15 days of the trace period (the first 4320 datapoints of the trace).

In [ ]:
%%bigquery mvpp1_df
SELECT
  time,
  measured_power_util,
  production_power_util
FROM `google.com:google-cluster-data`.powerdata_2019.celli_mvpp1
ORDER BY time
LIMIT 4320

In [ ]:
alt.Chart(mvpp1_df).mark_line().transform_fold(
    ["measured_power_util", "production_power_util"]).encode(
    x="time:Q",
    y=alt.X("value:Q", scale=alt.Scale(zero=False)),
    color="key:N"
).properties(width=700, height=75)

## CPU and Power over Time

As an example of joining the power traces with the cluster traces, we'll plot the average CPU utilization and power utilization per hour.

You may remember this query from the [cluster analysis colab](https://github.com/google/cluster-data/blob/master/clusterdata_analysis_colab.ipynb). It's been modified slightly--see `query_per_tier_utilization_time_series` in particular.

In [ ]:
def machines_in_pdu(pdu_number):
  return '''
SELECT machine_id
FROM `google.com:google-cluster-data`.powerdata_2019.machine_to_pdu_mapping
WHERE pdu = 'pdu{pdu_number}'
  '''.format(pdu_number=pdu_number)

def query_cell_capacity(cell, pdu_number):
  return '''
SELECT SUM(cpu_cap) AS cpu_capacity
FROM (
  SELECT machine_id, MAX(capacity.cpus) AS cpu_cap,
  FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.machine_events
  WHERE machine_id IN ({machine_query})
  GROUP BY 1
)
  '''.format(cell=cell, machine_query=machines_in_pdu(pdu_number))

def query_per_instance_usage_priority(cell, pdu_num):
  return '''
SELECT u.time AS time,
  u.collection_id AS collection_id,
  u.instance_index AS instance_index,
  e.priority AS priority,
  CASE
    WHEN e.priority BETWEEN 0 AND 99 THEN '1_free'
    WHEN e.priority BETWEEN 100 AND 115 THEN '2_beb'
    WHEN e.priority BETWEEN 116 AND 119 THEN '3_mid'
    ELSE '4_prod'
  END AS tier,
  u.cpu_usage AS cpu_usage
FROM (
  SELECT start_time AS time,
    collection_id,
    instance_index,
    machine_id,
    average_usage.cpus AS cpu_usage
  FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.instance_usage
  WHERE (alloc_collection_id IS NULL OR alloc_collection_id = 0)
    AND (end_time - start_time) >= (5 * 60 * 1e6)
) AS u JOIN (
  SELECT collection_id, instance_index, machine_id,
    MAX(priority) AS priority
  FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.instance_events
  WHERE (alloc_collection_id IS NULL OR alloc_collection_id = 0)
    AND machine_id IN ({machine_query})
  GROUP BY 1, 2, 3
) AS e ON u.collection_id = e.collection_id
  AND u.instance_index = e.instance_index
  AND u.machine_id = e.machine_id
  '''.format(cell=cell, machine_query=machines_in_pdu(pdu_num))

def query_per_tier_utilization_time_series(cell, pdu_num, cpu_capacity):
  return '''
SELECT * FROM (
  SELECT CAST(FLOOR(time/(1e6 * 60 * 60)) AS INT64) AS hour_index,
    tier,
    SUM(cpu_usage) / (12 * {cpu_capacity}) AS avg_cpu_usage,
  FROM ({table})
  GROUP BY 1, 2)
JOIN (
  SELECT CAST(FLOOR(time/(1e6 * 60 * 60)) AS INT64) AS hour_index,
  pdu,
  AVG(measured_power_util) as avg_measured_power_util,
  AVG(production_power_util) AS avg_production_power_util
  FROM `google.com:google-cluster-data`.`powerdata_2019.cell{cell}_pdu{pdu_num}`
  GROUP BY hour_index, pdu
) USING (hour_index)
  '''.format(table=query_per_instance_usage_priority(cell, pdu_num),
             cpu_capacity=cpu_capacity, cell=cell, pdu_num=pdu_num)

def run_query_utilization_per_time_time_series(cell, pdu_num):
  cell_cap = client.query(query_cell_capacity(cell, pdu_num)).to_dataframe()
  query = query_per_tier_utilization_time_series(
      cell,
      pdu_num,
      cell_cap['cpu_capacity'][0])
  time_series = client.query(query).to_dataframe()
  return time_series

CELL='f'
PDU_NUM='17'
hourly_usage = run_query_utilization_per_time_time_series(CELL, PDU_NUM)

Plot power utilization on top of the CPU utilization graph.

In [ ]:
# CPU graph
cpu = alt.Chart().mark_area().encode(
        alt.X('hour_index:N'),
        alt.Y('avg_cpu_usage:Q'),
        color=alt.Color('tier', legend=alt.Legend(orient="left", title=None)),
        order=alt.Order('tier', sort='descending'),
        tooltip=['tier:N', 'avg_cpu_usage:Q']
  )
cpu.encoding.x.title = "Hour"
cpu.encoding.y.title = "Average Utilization"


# Power Utilization graph
pu = (
    alt.Chart()
    .transform_fold(['avg_measured_power_util', 'avg_production_power_util'])
    .encode(
        alt.X(
            'hour_index:N',
            axis=alt.Axis(labels=False, domain=False, ticks=False),
        ),
        alt.Y('value:Q'),
        color=alt.Color('key:N', legend=None),
        strokeDash=alt.StrokeDash('key:N', legend=None),
        tooltip=['hour_index:N', 'key:N', 'value:Q']
    )
    .mark_line().properties(title=alt.datum.cell + ': ' + alt.datum.cluster)
)


alt.layer(cpu, pu, data=hourly_usage).properties(
    width=1200,
    height=300,
    title="Average CPU and Power Utilization").configure_axis(grid=False)

We can adapt the previous queries to calculate the average CPU and power utilizations per day by tier (i.e. cappable workloads or production) for each PDU in cell `b`.

In [ ]:
%%bigquery cluster_and_power_data_df
WITH
  machines_in_cell AS (
    SELECT machine_id, pdu, cell
    FROM `google.com:google-cluster-data`.powerdata_2019.machine_to_pdu_mapping
    WHERE cell = 'b'
  ),
  cpu_capacities AS (
    SELECT pdu, cell, SUM(cpu_cap) AS cpu_capacity
    FROM
      (
        SELECT machine_id, MAX(capacity.cpus) AS cpu_cap,
        FROM `google.com:google-cluster-data`.clusterdata_2019_b.machine_events
        GROUP BY 1
      )
    JOIN machines_in_cell
      USING (machine_id)
    GROUP BY 1, 2
  ),
  per_instance_usage_priority AS (
    SELECT
      u.time AS time,
      u.collection_id AS collection_id,
      u.instance_index AS instance_index,
      e.priority AS priority,
      IF(e.priority < 120, 'cappable', 'production') AS tier,
      u.cpu_usage AS cpu_usage,
      m.pdu
    FROM
      (
        SELECT
          start_time AS time,
          collection_id,
          instance_index,
          machine_id,
          average_usage.cpus AS cpu_usage
        FROM `google.com:google-cluster-data`.clusterdata_2019_b.instance_usage
        WHERE
          (alloc_collection_id IS NULL OR alloc_collection_id = 0)
          AND (end_time - start_time) >= (5 * 60 * 1e6)
      ) AS u
    JOIN
      (
        SELECT collection_id, instance_index, machine_id, MAX(priority) AS priority
        FROM `google.com:google-cluster-data`.clusterdata_2019_b.instance_events
        WHERE (alloc_collection_id IS NULL OR alloc_collection_id = 0)
        GROUP BY 1, 2, 3
      ) AS e
      ON
        u.collection_id = e.collection_id
        AND u.instance_index = e.instance_index
        AND u.machine_id = e.machine_id
    JOIN machines_in_cell AS m
      ON m.machine_id = u.machine_id
  )
SELECT *
FROM
  (
    SELECT
      CAST(FLOOR(time / (1e6 * 60 * 60 * 24)) AS INT64) AS day_index,
      pdu,
      tier,
      SUM(cpu_usage) / (12 * 24 * ANY_VALUE(cpu_capacity)) AS avg_cpu_usage,
    FROM per_instance_usage_priority
    JOIN cpu_capacities
      USING (pdu)
    GROUP BY 1, 2, 3
  )
JOIN
  (
    SELECT
      CAST(FLOOR((time - 6e8 + 3e8) / (1e6 * 60 * 60 * 24)) AS INT64) AS day_index,
      pdu,
      AVG(measured_power_util) AS avg_measured_power_util,
      AVG(production_power_util) AS avg_production_power_util
    FROM `google.com:google-cluster-data`.`powerdata_2019.cellb_pdu*`
    GROUP BY 1, 2
  )
  USING (pdu, day_index)

In [ ]:
cluster_and_power_data_df.describe()

### Be careful joining on `time` directly!
Note that in the queries above we converted `time` from the cluster and power datasets to an `hour_index` which we then use to join the two datasets. We don't want to join on `time` directly due to the how the datasets are structured.

The power trace `time` values are each aligned to the 5-minute mark. For example, there's a `time` at 600s, 900s, 1200s but never 700s or 601s. The cluster trace `time` values have no specific alignment. If we were to join the two datasets on `time`, we'd end up dropping a lot of data!

You may have also noticed that there are `day_index` has 32 unique values in the query above, despite May having 31 days. This is because timestamps in the data sets are represented as microseconds since 600 seconsd before the start of the trace period, May 01 2019 at 00:00 PT.

## Recreating Graphs from the Paper

Below, the power traces are used to re-create figures from [the paper](https://research.google/pubs/pub49032/).

In [ ]:
def histogram_data(filter='pdu%', agg_by='pdu', util_type='measured_power_util'):
  query = """
  SELECT bin as bins, SUM(count) as counts
  FROM (
    SELECT {agg_by},
           ROUND(CAST({util_type} / 0.0001 as INT64) * 0.0001, 3) as bin,
           COUNT(*) as count
    FROM (
      SELECT {agg_by},
             time,
             {util_type},
      FROM `google.com:google-cluster-data`.`powerdata_2019.cell*`
      WHERE NOT bad_measurement_data
            AND NOT bad_production_power_data
            AND {agg_by} LIKE '{filter}'
            AND NOT cell in ('i', 'j')
    )
    GROUP BY 1, 2
  ) GROUP BY 1 ORDER BY 1;
  """.format(**{'filter': filter, 'agg_by': agg_by, 'util_type': util_type})
  return client.query(query).to_dataframe()

In [ ]:
pdu_df = histogram_data()
cluster_df = histogram_data('%', 'cell')
prod_pdu_df = histogram_data(util_type='production_power_util')
prod_cluster_df = histogram_data('%', 'cell', util_type='production_power_util')

In [ ]:
def make_cdf(p_df, c_df, title):
  pdu_counts, pdu_bins = p_df.counts, p_df.bins
  cluster_counts, cluster_bins = c_df.counts, c_df.bins

  pdu_cdf = np.cumsum (list(pdu_counts))
  pdu_cdf = (1.0 * pdu_cdf) / pdu_cdf[-1]
  cluster_cdf = np.cumsum (list(cluster_counts))
  cluster_cdf = (1.0 * cluster_cdf) / cluster_cdf[-1]

  pdu_cdf_graph = alt.Chart(pd.DataFrame(
      {'bins': pdu_bins, 'cdf': pdu_cdf})).mark_point(size=.1).encode(
        x=alt.X('bins', scale=alt.Scale(domain=[0.4, 0.90])),
        y=alt.Y('cdf'),
        color=alt.value('steelblue')
  )

  cluster_cdf_graph = alt.Chart(pd.DataFrame(
      {'bins': cluster_bins, 'cdf': cluster_cdf})).mark_point(size=.1).encode(
        x=alt.X('bins', scale=alt.Scale(domain=[0.4, 0.90])),
        y=alt.Y('cdf'),
        color=alt.value('forestgreen')
  )

  return (pdu_cdf_graph + cluster_cdf_graph).properties(title=title)


alt.hconcat(make_cdf(pdu_df, cluster_df, "Measured Power Util"), make_cdf(
    prod_pdu_df, prod_cluster_df, "Production Power Util"))